In [3]:
import numpy as np
from cmath import * 
import torch
import pickle
import os

In [4]:
from ipynb.fs.full.Rough_Bergomi import *

In [6]:
# Rough Bergomi Parameters

V_0=0.235**2 #Forward Variance
nu=1.9 
rho=-0.7 # Correlation
S_0=100 # Spot price
H=0.1 # Hurst indice 
N_Disc=1000 # Discretization


# Product to Hedge Parameters
Product="European Call"
T=30/365 # Maturity
K=100 #strike
time_steps=30 # Daily Hedging

def Call_Payoff(S,K):
    return(np.maximum(S-K,0))


# Neural Network Training Parameters

batch_size=256
path_training=30*batch_size
vect_days=np.round(np.linspace(0,N_Disc-1,30)).astype(int) # days indexes


In [7]:
#Generate training data

training_set=int( (10**6)/(path_training)) # training set size

F1=fBM(path_training, N_Disc, T, H, rho)
B1=rBergomi(path_training, N_Disc, T , H , V_0, nu, S_0,rho,F1)
S1_train=B1.S()
S1_train=S1_train[:,vect_days]
V1_train=B1.V()
V1_train=V1_train[:,vect_days]

p_0=np.ones((path_training,time_steps,1))*np.mean(Call_Payoff(S1_train[:,-1],K))
p_0=torch.Tensor(p_0)
S1_train=torch.reshape(torch.Tensor(S1_train),(S1_train.shape[0],S1_train.shape[1],1))
V1_train=torch.reshape(torch.Tensor(V1_train),(V1_train.shape[0],V1_train.shape[1],1))

for set_ in range (training_set-1):
    
    F=fBM(path_training, N_Disc, T, H, rho)
    B=rBergomi(path_training, N_Disc, T , H , V_0, nu, S_0,rho,F)
    S_train=B.S()
    V_train=B.V()
    V_train=V_train[:,vect_days]
    S_train=S_train[:,vect_days]
    p_=np.ones((path_training,time_steps,1))*np.mean(Call_Payoff(S_train[:,-1],K))
    p_=torch.Tensor(p_)
    S_train=torch.reshape(torch.Tensor(S_train),(S_train.shape[0],S_train.shape[1],1))
    V_train=torch.reshape(torch.Tensor(V_train),(V_train.shape[0],V_train.shape[1],1))
    V1_train=torch.cat((V1_train,V_train),dim=0)
    S1_train=torch.cat((S1_train,S_train),dim=0)
    p_0=torch.cat((p_0,p_),dim=0)

print('S1_train Shape', S1_train.shape)


S1_train Shape torch.Size([7680, 30, 1])


In [8]:
#Save training data 

S1_training_File = "S1_training_data.pkl"  
p_0_training_File = "p_0_training_data.pkl"  
V1_training_File= "V1_training_data.pkl"

with open(S1_training_File, 'wb') as file:  
    pickle.dump(S1_train, file)
    
with open(p_0_training_File, 'wb') as file:  
    pickle.dump(p_0, file)
    
with open(V1_training_File, 'wb') as file:  
    pickle.dump(V1_train, file)